##  Predicting Baseball Hall of Fame Induction

###  Dataset Assembly

###  Eryk Wdowiak and Ken Hoffman

data from Lahman Baseball Database

### load libraries and data

In [1]:
import warnings
# warnings.filterwarnings('ignore',category=RuntimeWarning)

import numpy as np
import pandas as pd

import pickle

In [2]:
!ls baseballdatabank-master/core/

AllstarFull.csv		 FieldingOF.csv       PitchingPost.csv
Appearances.csv		 FieldingOFsplit.csv  readme2014.txt
AwardsManagers.csv	 FieldingPost.csv     Salaries.csv
AwardsPlayers.csv	 HallOfFame.csv       Schools.csv
AwardsShareManagers.csv  HomeGames.csv	      SeriesPost.csv
AwardsSharePlayers.csv	 Managers.csv	      Teams.csv
Batting.csv		 ManagersHalf.csv     TeamsFranchises.csv
BattingPost.csv		 Parks.csv	      TeamsHalf.csv
CollegePlaying.csv	 People.csv
Fielding.csv		 Pitching.csv


In [3]:
##  load appearances -- to determine eligibility
appears = pd.read_csv('baseballdatabank-master/core/Appearances.csv')

##  general infielder category
appears['G_if'] = appears['G_1b'] + appears['G_2b'] + appears['G_3b'] + appears['G_ss']

##  player tables -- main source of features
bat_reg = pd.read_csv('baseballdatabank-master/core/Batting.csv')
bat_pst = pd.read_csv('baseballdatabank-master/core/BattingPost.csv')
fld_reg = pd.read_csv('baseballdatabank-master/core/Fielding.csv')
fld_pst = pd.read_csv('baseballdatabank-master/core/FieldingPost.csv')
ptc_reg = pd.read_csv('baseballdatabank-master/core/Pitching.csv')
ptc_pst = pd.read_csv('baseballdatabank-master/core/PitchingPost.csv')
hll_fme = pd.read_csv('baseballdatabank-master/core/HallOfFame.csv')

##  team post season statistics
ser_pst = pd.read_csv('baseballdatabank-master/core/SeriesPost.csv')

###  which players are eligible?

In [4]:
##  minimum criteria:  five years since retirement and played at least ten seasons
##  also, let's only look players who retired after World War Two

##  first, count seasons
seasons = appears[['yearID','playerID']].groupby('playerID')['yearID'].nunique()
seasons_dict = dict(zip(list(seasons.index), list(seasons.values)))
appears['nu_seasons'] = appears['playerID'].apply(lambda x: seasons_dict[x])
del seasons

##  now get last season
last_season = appears[['yearID','playerID']].groupby('playerID')['yearID'].max()
last_dict = dict(zip(list(last_season.index), list(last_season.values)))
appears['last_season'] = appears['playerID'].apply(lambda x: last_dict[x])
del last_season

##  eligibles
eligibles = appears[((appears['last_season'] > 1945) & 
                     (appears['last_season'] <= 2013) & 
                     (appears['nu_seasons'] >= 10 ))]

##  get list of eligible players
list_elig = list(eligibles['playerID'].unique())

In [5]:
##  limit the player tables to the eligibles
bat_reg = bat_reg[bat_reg['playerID'].isin(list_elig)]
bat_pst = bat_pst[bat_pst['playerID'].isin(list_elig)]
fld_reg = fld_reg[fld_reg['playerID'].isin(list_elig)]
fld_pst = fld_pst[fld_pst['playerID'].isin(list_elig)]
ptc_reg = ptc_reg[ptc_reg['playerID'].isin(list_elig)]
ptc_pst = ptc_pst[ptc_pst['playerID'].isin(list_elig)]
hll_fme = hll_fme[hll_fme['playerID'].isin(list_elig)]

In [6]:
##  get list of inductees
list_induct = list(hll_fme['playerID'][(hll_fme['inducted']=='Y') & 
                                       (hll_fme['category']=='Player')])

##  create induction dictionary
list_zero = [0] * len(list_elig)
indc_dict = dict(zip(list_elig,list_zero))
for playerID in list_induct:
    indc_dict[playerID] = 1
del list_zero

###  what position did they play?

In [7]:
##  columns of interest
cols_int = ['playerID','G_all','GS','G_batting','G_defense',
            'G_p','G_c','G_if', #'G_1b','G_2b','G_3b','G_ss',
            'G_of', #'G_lf','G_cf','G_rf',
            'G_dh','G_ph','G_pr']

##  sum appearances at each position
positions = eligibles[cols_int].groupby('playerID').sum()
del cols_int

##  position played most often
positions['pos_most'] = positions[['G_p','G_c','G_if','G_of']].idxmax(axis='columns')

##  count percentage of times at that position
positions['pos_count'] = positions[['G_p','G_c','G_if','G_of']].max(axis='columns')
positions['pos_sum'] = positions[['G_p','G_c','G_if','G_of']].sum(axis='columns')
positions['pos_pct'] = positions['pos_count'] / positions['pos_sum']

##  give better names to the categories
gpos_dict = {'G_p':'pitcher','G_c':'catcher','G_if':'infielder','G_of':'outfielder'}
positions['pos_most'] = positions['pos_most'].apply(lambda x: gpos_dict[x])
del gpos_dict

##  create position dictionary
postn_dict = dict(zip(list(positions.index),list(positions['pos_most'])))

##  now create lists for each of the four major positions
pitchers = list(positions[(positions['pos_most'] == 'pitcher')].index)
catchers = list(positions[(positions['pos_most'] == 'catcher')].index)
infielders = list(positions[(positions['pos_most'] == 'infielder')].index)
outfielders = list(positions[(positions['pos_most'] == 'outfielder')].index)

###  what team did they play for?

In [8]:
##  columns of interest
cols_int = ['teamID','playerID','G_all']

##  sum appearances by team
team_df = eligibles[cols_int].groupby(['playerID','teamID']).sum()

##  where did they play most?
team_list = list(team_df.groupby('playerID')['G_all'].idxmax().values)

##  create team dictionary
pl_list = [team[0] for team in team_list]
tm_list = [team[1] for team in team_list]
team_dict = dict(zip(pl_list,tm_list))

## clean up
del cols_int, team_list, team_df, pl_list, tm_list

### which pitchers go to hall of fame?

In [9]:
##  limit the pitcher tables to the pitchers
ptc_reg = ptc_reg[ptc_reg['playerID'].isin(pitchers)]
ptc_pst = ptc_pst[ptc_pst['playerID'].isin(pitchers)]

##  get rid of the ones who recorded no outs
ptc_reg = ptc_reg[(ptc_reg['IPouts'] > 0)]
ptc_pst = ptc_pst[(ptc_pst['IPouts'] > 0)]

##  get rid of problematic columns that we don't need
ptc_reg = ptc_reg.drop(columns=['IBB','SH','SF','GIDP'])
ptc_pst = ptc_pst.drop(columns=['IBB','SH','SF','GIDP'])

##  we can recalculate ERA and BAOpp later if we need them
ptc_reg = ptc_reg.drop(columns=['ERA','BAOpp'])
ptc_pst = ptc_pst.drop(columns=['ERA','BAOpp'])

In [10]:
##  columns to sum
cols_int = ['playerID','W','L','G','GS','CG','SHO','SV','IPouts',
            'H','ER','HR','BB','SO','WP','HBP','BK','BFP','GF','R']

##  sum the columns by player
ptc_reg = ptc_reg[cols_int].groupby('playerID').sum().reset_index()
ptc_pst = ptc_pst[cols_int].groupby('playerID').sum().reset_index()
del cols_int

##  join them
ptc_df = ptc_reg.join(ptc_pst.set_index('playerID'),
                      on='playerID',rsuffix='_ps').fillna(0)

##  add info
ptc_df['nu_sns'] = ptc_df['playerID'].apply(lambda x: seasons_dict[x])
ptc_df['lst_sn'] = ptc_df['playerID'].apply(lambda x: last_dict[x])
ptc_df['induct'] = ptc_df['playerID'].apply(lambda x: indc_dict[x])
ptc_df['teamID'] = ptc_df['playerID'].apply(lambda x: team_dict[x])

In [11]:
##  let's take a look
ptc_df[['playerID','nu_sns','lst_sn','teamID','induct',
        'W','L','G','GS','IPouts',
        'W_ps','L_ps','G_ps','GS_ps','IPouts_ps']].tail(10)

,playerID,nu_sns,lst_sn,teamID,induct,W,L,G,GS,IPouts,W_ps,L_ps,G_ps,GS_ps,IPouts_ps
890,wrighcl01,10,1975,CAL,0,100,111,329,235,5186,0.0,0.0,0.0,0.0,0.0
891,wrighja02,11,2007,CLE,0,68,60,226,171,2918,3.0,6.0,16.0,10.0,168.0
892,wynnea01,23,1963,CLE,1,300,244,691,612,13692,1.0,2.0,4.0,4.0,60.0
893,yanes01,11,2006,TBA,0,33,39,472,23,2086,0.0,0.0,1.0,0.0,6.0
894,youngcu01,11,1993,OAK,0,69,53,251,162,3321,0.0,0.0,3.0,0.0,10.0
895,youngma01,10,1993,SEA,0,55,95,333,163,3569,0.0,0.0,1.0,0.0,1.0
896,zachrpa01,10,1985,NYN,0,69,67,293,154,3532,2.0,0.0,4.0,2.0,47.0
897,zahnge01,13,1985,MIN,0,111,109,304,270,5547,0.0,1.0,1.0,1.0,11.0
898,zambrca01,12,2012,CHN,0,132,91,354,302,5877,0.0,2.0,5.0,5.0,87.0
899,zuberbi01,11,1947,WS1,0,43,42,224,65,2358,0.0,0.0,1.0,0.0,6.0


###  which fielders go to hall of fame?

In [12]:
##  get list of fielders
fielders = catchers + infielders + outfielders

##  limit the batting tables to fielders
bat_reg = bat_reg[bat_reg['playerID'].isin(fielders)]
bat_pst = bat_pst[bat_pst['playerID'].isin(fielders)]

##  get rid of problematic columns that we don't need
##  missing 903 "caught stealing values" in regular season
bat_reg = bat_reg.drop(columns=['IBB','SF','GIDP']) ##,'CS'])
bat_pst = bat_pst.drop(columns=['IBB','SF','GIDP'])

##  limit the fielding tables to fielders
fld_reg = fld_reg[fld_reg['playerID'].isin(fielders)]
fld_pst = fld_pst[fld_pst['playerID'].isin(fielders)]

##  get rid of problematic columns that we don't need
##  note:  'SB' and 'CS' for catchers only
fld_reg = fld_reg.drop(columns=['GS','InnOuts','PB','WP','ZR']) ##,'SB','CS'])
fld_pst = fld_pst.drop(columns=['GS','InnOuts','PB']) ##,'SB','CS'])

In [13]:
##  columns to sum
cols_int = ['playerID','G','AB','R','H','2B','3B','HR',
            'RBI','SB','CS','BB','SO','HBP','SH']

##  sum the columns by player
bat_reg = bat_reg[cols_int].groupby('playerID').sum().reset_index()
bat_pst = bat_pst[cols_int].groupby('playerID').sum().reset_index()
del cols_int

##  join them
bat_df = bat_reg.join(bat_pst.set_index('playerID'),
                      on='playerID',rsuffix='_ps').fillna(0)

##  rename batting stat names that overlap with fielding stat names
bat_df = bat_df.rename(columns={'G':'bG','SB':'bSB','CS':'bCS',
                                'G_ps':'bG_ps','SB_ps':'bSB_ps','CS_ps':'bCS_ps',})

##  add info
# bat_df['nu_sns'] = bat_df['playerID'].apply(lambda x: seasons_dict[x])
# bat_df['lst_sn'] = bat_df['playerID'].apply(lambda x: last_dict[x])
# bat_df['induct'] = bat_df['playerID'].apply(lambda x: indc_dict[x])
# bat_df['teamID'] = bat_df['playerID'].apply(lambda x: team_dict[x])
# bat_df['position'] = bat_df['playerID'].apply(lambda x: postn_dict[x])

In [14]:
##  let's take a look
bat_df[['playerID',#'nu_sns','lst_sn','teamID','position','induct',
        'bG','AB','R','H','bG_ps','AB_ps','R_ps','H_ps']].head()

,playerID,bG,AB,R,H,bG_ps,AB_ps,R_ps,H_ps
0,aaronha01,3298,12364,2174,3771,17.0,69.0,11.0,25.0
1,adairje01,1165,4019,378,1022,5.0,16.0,0.0,2.0
2,adamsbo03,1281,4019,591,1082,0.0,0.0,0.0,0.0
3,adcocjo01,1959,6606,823,1832,9.0,28.0,2.0,7.0
4,ageeto01,1129,3912,558,999,8.0,32.0,5.0,8.0


In [15]:
##  columns to sum
cols_int = ['playerID','G','PO','A','E','DP','SB','CS']

##  sum the columns by player
fld_reg = fld_reg[cols_int].groupby('playerID').sum().reset_index()
fld_pst = fld_pst[cols_int].groupby('playerID').sum().reset_index()
del cols_int

##  join them
fld_df = fld_reg.join(fld_pst.set_index('playerID'),
                      on='playerID',rsuffix='_ps').fillna(0)

##  rename fielding stat names that overlap with batting stat names
fld_df = fld_df.rename(columns={'G':'fG','SB':'fSB','CS':'fCS',
                                'G_ps':'fG_ps','SB_ps':'fSB_ps','CS_ps':'fCS_ps',})

##  add info
# fld_df['nu_sns'] = fld_df['playerID'].apply(lambda x: seasons_dict[x])
# fld_df['lst_sn'] = fld_df['playerID'].apply(lambda x: last_dict[x])
# fld_df['induct'] = fld_df['playerID'].apply(lambda x: indc_dict[x])
# fld_df['teamID'] = fld_df['playerID'].apply(lambda x: team_dict[x])
# fld_df['position'] = fld_df['playerID'].apply(lambda x: postn_dict[x])

In [16]:
##  let's take a look
fld_df[['playerID',#'nu_sns','lst_sn','teamID','position','induct',
        'fG','PO','fG_ps','PO_ps']].head()

,playerID,fG,PO,fG_ps,PO_ps
0,aaronha01,3020,7436,18.0,31.0
1,adairje01,1167,2380,4.0,7.0
2,adamsbo03,1066,1748,0.0,0.0
3,adcocjo01,1811,13678,8.0,61.0
4,ageeto01,1073,2371,8.0,28.0


In [17]:
##  now join batting and fielding statistics
bat_fld = bat_df.join(fld_df.set_index('playerID'),on='playerID').fillna(0)

##  add info
bat_fld['nu_sns'] = bat_fld['playerID'].apply(lambda x: seasons_dict[x])
bat_fld['lst_sn'] = bat_fld['playerID'].apply(lambda x: last_dict[x])
bat_fld['induct'] = bat_fld['playerID'].apply(lambda x: indc_dict[x])
bat_fld['teamID'] = bat_fld['playerID'].apply(lambda x: team_dict[x])
bat_fld['position'] = bat_fld['playerID'].apply(lambda x: postn_dict[x])

In [18]:
##  let's take a look
bat_fld[['playerID','nu_sns','lst_sn','teamID','position','induct',
         'bG','bG_ps','fG','fG_ps']].head()

,playerID,nu_sns,lst_sn,teamID,position,induct,bG,bG_ps,fG,fG_ps
0,aaronha01,23,1976,ML1,outfielder,1,3298,17.0,3020,18.0
1,adairje01,13,1970,BAL,infielder,0,1165,5.0,1167,4.0
2,adamsbo03,14,1959,CIN,infielder,0,1281,0.0,1066,0.0
3,adcocjo01,17,1966,ML1,infielder,0,1959,9.0,1811,8.0
4,ageeto01,12,1973,NYN,outfielder,0,1129,8.0,1073,8.0


### store tables for analysis

In [19]:
##  split into dataframes by position
catchers_df    = bat_fld[bat_fld['position']=='catcher']
infielders_df  = bat_fld[bat_fld['position']=='infielder']
outfielders_df = bat_fld[bat_fld['position']=='outfielder']

##  get rid of fielding 'SB' and 'CS' which are for catchers only
infielders_df  =  infielders_df.drop(columns=['fSB','fCS','fSB_ps','fCS_ps'])
outfielders_df = outfielders_df.drop(columns=['fSB','fCS','fSB_ps','fCS_ps'])

##  get pitchers dataframe
pitchers_df = ptc_df.copy()

In [20]:
##  create a little date string
dt_str = '2020-08-19a'

##  save them to files
pickle.dump(pitchers_df,open('pitchers-df_'+ dt_str +'.p','wb'))
pickle.dump(catchers_df,open('catchers-df_'+ dt_str +'.p','wb'))
pickle.dump(infielders_df,open('infielders-df_'+ dt_str +'.p','wb'))
pickle.dump(outfielders_df,open('outfielders-df_'+ dt_str +'.p','wb'))

##  Continue on!

This concludes our assembly of the baseball hall of fame data.  
In our next notebook, we conduct a statistical analysis of the data.